In [172]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [173]:
# Checking df column datatypes
df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
School ID         int64
type             object
size              int64
budget            int64
dtype: object

In [174]:
# Check for null values
df.isna().sum()

Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
School ID        0
type             0
size             0
budget           0
dtype: int64

In [175]:
# Gettting a look at the data structure
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [176]:
# Calculate total number of schools
total_schools = school_data['school_name'].nunique()

In [177]:
# Calculate total number of students
total_students = school_data['size'].sum()

In [178]:
# Calculate total budget
total_budget = school_data['budget'].sum()

In [179]:
# Calculate average math score
average_math_score = df['math_score'].mean()

In [180]:
# Calculate average reading score
average_reading_score = df['reading_score'].mean()

In [287]:
# Calculate percentage of students that have a passing math score
percent_pass_math = (df['math_score'][df.math_score >= 70].count() / len(df['math_score'])) * 100

In [288]:
# Calculate percentage of students that have a passing reading score
percent_pass_reading = (df['reading_score'][df.reading_score >= 70].count() / len(df['reading_score'])) * 100

In [289]:
# Calculate percentage of students that passed overall
percent_pass_overall = (df['Student ID'][(df.reading_score >= 70) & (df.math_score >= 70)].count() / len(df['Student ID']) * 100)

In [290]:
# Create a dataframe to hold the results
district_summary = pd.DataFrame({
    "Total Schools" : total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": percent_pass_math,
    "% Passing Reading": percent_pass_reading,
    "% Overall Passing": percent_pass_overall
}, index = [0])

# District Summary

In [291]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [302]:
# Create a DataFrame to be joined on by calculations
school_summary = pd.DataFrame({
    "School Name": school_data['school_name'],
    "School Type": school_data['type'],
    "Total Budget": school_data['budget']
})

In [260]:
# Calculate budget per student
for school in school_data['school_name']:
    budget_per_student = pd.DataFrame({"School Name": school_data['school_name'],
                                       "Budget Per Student" : school_data['budget'] / school_data['size']})

In [261]:
# Create DataFrame of just school names and reading scores
school_reading_average_score = df[['school_name','reading_score']]

# Group this DataFrame by school and aggregate by reading score average
school_reading_average_score = school_reading_average_score.groupby(['school_name']).mean()

# Reset index to default
school_reading_average_score = school_reading_average_score.reset_index()

# Rename column
school_reading_average_score = school_reading_average_score.rename(columns={"school_name": "School Name"})

In [262]:
# Create DataFrame of just school names and math scores
school_math_average_score = df[['school_name','math_score']]

# Group this DataFrame by school and aggregate by math score average
school_math_average_score = school_math_average_score.groupby(['school_name']).mean()

# Reset index to default
school_math_average_score = school_math_average_score.reset_index()

# Rename column
school_math_average_score = school_math_average_score.rename(columns={"school_name": "School Name"})

In [263]:
# Create DataFrame of just school names and reading scores
school_reading_pass_rate = df[['school_name', 'reading_score']]

# Rename school_name to better formatting
school_reading_pass_rate = school_reading_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_reading_pass_rate['student_count'] = 1

# Create column pass/fail for conditional >/< 70
school_reading_pass_rate['pass_count'] = [1 if score > 70 else 0 for score in df['reading_score']]

# Group by school
school_reading_pass_rate = school_reading_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_reading_pass_rate['% Passing Reading'] = (
    school_reading_pass_rate['pass_count'] / school_reading_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_reading_pass_rate = school_reading_pass_rate.drop(
    ['reading_score','student_count','pass_count'], 1)

# Reset index
school_reading_pass_rate = school_reading_pass_rate.reset_index()

/var/folders/_q/q0k0vyk516z58q89cytv2_380000gn/T/ipykernel_29964/1119901948.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  school_reading_pass_rate = school_reading_pass_rate.drop(


In [264]:
# Create DataFrame of just school names and math scores
school_math_pass_rate = df[['school_name', 'math_score']]

# Rename school_name to better formatting
school_math_pass_rate = school_math_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_math_pass_rate['student_count'] = 1

# Create column pass/fail for conditional >/< 70
school_math_pass_rate['pass_count'] = [1 if score > 70 else 0 for score in df['math_score']]

# Group by school
school_math_pass_rate = school_math_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_math_pass_rate['% Passing Math'] = (
    school_math_pass_rate['pass_count'] / school_math_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_math_pass_rate = school_math_pass_rate.drop(
    ['math_score','student_count','pass_count'], 1)

/var/folders/_q/q0k0vyk516z58q89cytv2_380000gn/T/ipykernel_29964/3977246586.py:21: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  school_math_pass_rate = school_math_pass_rate.drop(


In [293]:
# Create DataFrame of just school names and reading scores
school_overall_pass_rate = df[['school_name', 'reading_score', 'math_score']]

# Rename school_name to better formatting
school_overall_pass_rate = school_overall_pass_rate.rename(columns={'school_name': 'School Name'})

# Create column counting students
school_overall_pass_rate['student_count'] = 1

# Create reading column pass/fail for conditional >/< 70
school_overall_pass_rate['reading_pass_count'] = [1 if score >= 70 else 0 for score in df['reading_score']]

# Create reading column pass/fail for conditional >/< 70
school_overall_pass_rate['math_pass_count'] = [1 if score >= 70 else 0 for score in df['math_score']]

# Sum up reading and math columns
school_overall_pass_rate['sum_pass_count'] = school_overall_pass_rate['reading_pass_count'] + school_overall_pass_rate['math_pass_count']

# Create overall column pass/fail for conditional = 2
school_overall_pass_rate['overall_pass_count'] = [1 if score == 2 else 0 
                                                  for score in school_overall_pass_rate['sum_pass_count']]

# Group by school
school_overall_pass_rate = school_overall_pass_rate.groupby('School Name').sum()

# Create pass fail percentage column
school_overall_pass_rate['% Passing Overall'] = (
    school_overall_pass_rate['overall_pass_count'] / school_overall_pass_rate['student_count'] * 100)

# Drop unnecessary columns
school_overall_pass_rate = school_overall_pass_rate.drop(
    ['reading_score', 'math_score', 'student_count', 'reading_pass_count', 'math_pass_count',
    'sum_pass_count', 'overall_pass_count'], 1)

/var/folders/_q/q0k0vyk516z58q89cytv2_380000gn/T/ipykernel_29964/2245393392.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  school_overall_pass_rate = school_overall_pass_rate.drop(


In [303]:
# Merge dataframes clean up extra columns
school_summary = pd.merge(school_summary, budget_per_student, how='outer', on=["School Name", "School Name"])

In [304]:
# Merge w/ average reading score
school_summary = pd.merge(school_summary, school_reading_average_score, how='outer', on=["School Name", "School Name"])
school_summary = school_summary.rename(columns={"reading_score": "Average Reading Score"})

In [305]:
# Merge w/ average math score
school_summary = pd.merge(school_summary, school_math_average_score, how='outer', on=["School Name", "School Name"])
school_summary = school_summary.rename(columns={"math_score": "Average Math Score"})

In [306]:
# Merge w/ % passing reading
school_summary = pd.merge(school_summary, school_reading_pass_rate, how='outer', on=["School Name", "School Name"])

In [308]:
# Merge w/ % passing math
school_summary = pd.merge(school_summary, school_math_pass_rate, how='outer', on=["School Name", "School Name"])

In [309]:
# Merge w/ % passing overall
school_summary = pd.merge(school_summary, school_overall_pass_rate, how='outer', on=["School Name", "School Name"])

# School Summary

In [310]:
school_summary

,School Name,School Type,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
0,Huang High School,District,1910635,655.0,81.182722,76.629414,78.813850,63.318478,53.513884
1,Figueroa High School,District,1884411,639.0,81.158020,76.711767,78.433367,63.750424,53.204476
2,Shelton High School,Charter,1056600,600.0,83.725724,83.359455,92.617831,89.892107,89.892107
3,Hernandez High School,District,3022020,652.0,80.934412,77.289752,78.187702,64.746494,53.527508
4,Griffin High School,Charter,917500,625.0,83.816757,83.351499,93.392371,89.713896,90.599455
5,Wilson High School,Charter,1319574,578.0,83.989488,83.274201,93.254490,90.932983,90.582567
6,Cabrera High School,Charter,1081356,582.0,83.975780,83.061895,93.864370,89.558665,91.334769
7,Bailey High School,District,3124928,628.0,81.033963,77.048432,79.300643,64.630225,54.642283
8,Holden High School,Charter,248087,581.0,83.814988,83.803279,92.740047,90.632319,89.227166
9,Pena High School,Charter,585858,609.0,84.044699,83.839917,92.203742,91.683992,90.540541


In [326]:
# Create a DataFrame of the top 5 rows by % Passing Overall
top_5_overall_passing = school_summary.nlargest(5,'% Passing Overall')
top_5_overall_passing.sort_values(by=['% Passing Overall'], inplace=True, ascending=False)

# Top Performing Schools (By % Overall Passing)

In [327]:
top_5_overall_passing

,School Name,School Type,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
6,Cabrera High School,Charter,1081356,582.0,83.975780,83.061895,93.864370,89.558665,91.334769
14,Thomas High School,Charter,1043130,638.0,83.848930,83.418349,92.905199,90.214067,90.948012
4,Griffin High School,Charter,917500,625.0,83.816757,83.351499,93.392371,89.713896,90.599455
5,Wilson High School,Charter,1319574,578.0,83.989488,83.274201,93.254490,90.932983,90.582567
9,Pena High School,Charter,585858,609.0,84.044699,83.839917,92.203742,91.683992,90.540541


In [328]:
# Create a DataFrame of the bottom 5 rows by % Passing Overall
bottom_5_overall_passing = school_summary.nsmallest(5,'% Passing Overall')
bottom_5_overall_passing.sort_values(by=['% Passing Overall'], inplace=True)

# Bottom Performing Schools (By % Overall Passing)

In [329]:
bottom_5_overall_passing

,School Name,School Type,Total Budget,Budget Per Student,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,% Passing Overall
11,Rodriguez High School,District,2547363,637.0,80.744686,76.842711,77.744436,64.066017,52.988247
1,Figueroa High School,District,1884411,639.0,81.158020,76.711767,78.433367,63.750424,53.204476
0,Huang High School,District,1910635,655.0,81.182722,76.629414,78.813850,63.318478,53.513884
3,Hernandez High School,District,3022020,652.0,80.934412,77.289752,78.187702,64.746494,53.527508
12,Johnson High School,District,3094650,650.0,80.966394,77.072464,78.281874,63.852132,53.539172
